パラメータの最適化
===========================

モデルとデータを用意できたので続いてはモデルを訓練、検証することで、データに対してモデルのパラメータを最適化し、テストを行う．

細かな説明は省くので何をしているのかいまいち思い出せない人は前のチュートリアルに戻るのもあり．

データセット
==============

前にやったデータセットを作るコードと同じ．

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

ネットワーク構築
================

同じ．

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

ハイパーパラメータ
==============

機械学習を進める上でネットワークの重み(パラメータ)やバイアスを最適化することになるが，よりうまく判別を行うために使い手（ぼくら）がイジる要素をハイパーパラメータと呼ぶ．

よく調整するハイパーパラメータは以下
 - **Epochs**：エポックの数．1エポックで一つのデータセットを一周した形になる．
 - **Batch Size**：ミニバッチサイズを構成するデータ数．大きいほど理論上分散は小さくなる
 - **Learning Rate**：パラメータ更新の係数．momentumが登場してから``0.1``とされることが多い．


In [1]:
epochs = 10
batch_size = 64
learning_rate = 0.1

"ここの値は一回終わった後に調整を推奨"
momentum = 0.0
weight_decay = 0.0

損失関数
================

一般的な損失関数としては、回帰タスクでは[`nn.MSELoss`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)(Mean Square Error)、分類タスクでは[`nn.NLLLoss`](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)(Negative Log Likelihood) が使用されることが多い．

[`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)は、``nn.LogSoftmax`` と ``nn.NLLLoss``を結合した損失関数．

モデルが出力する`logit`値を`nn.CrossEntropyLoss`に与えて正規化し、予測誤差を求めることになる

In [ ]:
loss = nn.CrossEntropyLoss()

torch.optim
================

使いたい最適化アルゴリズムをここで定義できる．

最適化アルゴリズムを自分で書いて定義する必要はない．

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum = momentum,weight_decay = weight_decay)

trainとtest
===================

最適化を行うtrainを予測を行うtestを実装する


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

実践
==============

これまで使ってきたコードを使って学習を行うことができる．

以下のコードを使って実際に精度を確認してみよう

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312256  [    0/60000]
loss: 1.165897  [ 6400/60000]
loss: 0.795653  [12800/60000]
loss: 0.862054  [19200/60000]
loss: 0.721507  [25600/60000]
loss: 0.667579  [32000/60000]
loss: 0.740483  [38400/60000]
loss: 0.789138  [44800/60000]
loss: 0.758234  [51200/60000]
loss: 0.724965  [57600/60000]
Test Error: 
 Accuracy: 77.5%, Avg loss: 0.011495 

Epoch 2
-------------------------------
loss: 0.665897  [    0/60000]
loss: 0.673182  [ 6400/60000]
loss: 0.616533  [12800/60000]
loss: 0.622670  [19200/60000]
loss: 0.585112  [25600/60000]
loss: 0.584928  [32000/60000]
loss: 0.615958  [38400/60000]
loss: 0.727573  [44800/60000]
loss: 0.674120  [51200/60000]
loss: 0.668467  [57600/60000]
Test Error: 
 Accuracy: 78.9%, Avg loss: 0.010828 

Epoch 3
-------------------------------
loss: 0.610003  [    0/60000]
loss: 0.611965  [ 6400/60000]
loss: 0.556257  [12800/60000]
loss: 0.545826  [19200/60000]
loss: 0.530171  [25600/60000]
loss: 0.554525  [32000/600

ハイパーパラメータを自分でいじって精度を上げてみるのもあり．

``momentum``や``weight_decay``は今は0で定義しているのでうまいこと変えてみよう！